In [40]:
import numpy as np
from sklearn.preprocessing import normalize
from sklearn.linear_model import Lasso
import scipy

In [172]:
def ols_estimations(X_lab, y):
    InverseMatrix = np.linalg.inv(np.dot(np.transpose(X_lab), X_lab))
    beta = np.dot(np.dot(InverseMatrix, np.transpose(X_lab)), y)
    return beta



def shooting_algorithm(A, X, y, lamda = 1, iterations = 100):
    beta = ols_estimations(X,y)
    n_lab = len(X)
    for iterations_number in range(iterations):
        for i in range(len(beta)):
            A_i = A[:, i]
            first_coef = np.linalg.norm(A_i) ** 2 
            second_coef = np.dot(y,  X[:, i]) / n_lab - np.dot(np.dot(A_i,  A[:, np.arange(5)!=i]), beta[np.arange(5)!=i])
            if (second_coef - lamda) > 0:
                beta[i] = (second_coef - lamda)/(first_coef)
            elif (second_coef + lamda) < 0:
                beta[i] = (second_coef + lamda)/(first_coef)
            else:
                beta[i] = 0
    return beta
            
    
def generate_linear_data(n = 100, test_n = 5000, dim = 5, sparse = False):

    mu = range(dim)
    Sigma = np.random.normal(10, 10, dim * dim)
    Sigma = np.reshape(Sigma, (dim, dim))
    Sigma = np.dot(np.transpose(Sigma), Sigma)
    X = np.random.multivariate_normal(mu, Sigma, size = n)
    X = normalize(X)
    y = np.sum(X, axis = 1) + np.random.normal(0, 1, n)
    X_test = np.random.multivariate_normal(mu, Sigma, size = test_n)
    X_test = normalize(X_test)
    y_test = np.sum(X_test, axis = 1) + np.random.normal(0, 1, test_n)
    return X,y,X_test, y_test


def shooting_algorithm_original(X, y, lamda = 1, iterations = 100):
    beta = ols_estimations(X,y)

    for iterations_number in range(iterations):
        for i in range(len(beta)):
            y_i = y - np.dot(X[:, np.arange(len(X[0,:]))!=i], beta[np.arange(len(beta))!=i])
            chislitel = np.dot(y_i, X[:, i])
            
            if chislitel - lamda > 0:
                beta[i] = (chislitel - lamda)/(np.dot(X[:, i],X[:, i]))
            elif chislitel + lamda < 0:
                beta[i] = (chislitel + lamda)/np.dot(X[:, i], X[:, i])
            else:
                beta[i] = 0
    loss = 1/2 * (np.linalg.norm(y - np.dot(X,beta)))**2 + lamda * np.linalg.norm(beta, ord = 1)
    print "Loss: " + str(loss)
    return beta

In [196]:
X,y, T, A = generate_linear_data(10000)

In [197]:
A = np.dot(np.transpose(X), X)/len(y)

In [198]:
A = scipy.linalg.sqrtm(A)

In [199]:
shooting_algorithm_original(X[:100, :], y[:100], lamda = 0.1, iterations=2000)

Loss: 0.501306143546


array([ 1.37811565,  1.00902726,  0.41902165,  1.35956209,  0.84733479])

In [214]:
shooting_algorithm(A, X[:100, :], y[:100], lamda = 0.001)

array([ 0.35698748,  1.25783096,  0.81168692,  1.90056271,  0.44109241])